In [19]:
import pandas as pd
import pyodbc
from datetime import datetime
import sqlalchemy

def process_mf_transactions(server, database, username, password):
    """
    Process MF order book and transactions data according to specified business rules.
    
    Args:
        server (str): SQL server name
        database (str): Database name
        username (str): Database username
        password (str): Database password
    """
    try:
        # Establish database connection
        conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
        # conn_str = sqlalchemy.create_engine(
                    # f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
                    #     )
        conn = pyodbc.connect(conn_str)
        # conn = conn_str
        
        # Step 1: Fetch unique regno entries from mf_order_book with status 'Accepted'
        order_book_query = """
        SELECT DISTINCT regno, investorcode, schemename, foliono, CAST(LEFT(regdate, 10) AS DATE) AS regdate
        FROM mf_order_book
        WHERE orderstatus = 'Accepted'
        """
        
        # Read data into pandas
        order_book_df = pd.read_sql(order_book_query, conn)
        
        if order_book_df.empty:
            print("No accepted orders found in mf_order_book")
            return pd.DataFrame()
        
        # Step 2: Preprocess folio numbers (take left part before '/')
        order_book_df['processed_folio'] = order_book_df['foliono'].apply(
            lambda x: x.split('/')[0] if x and '/' in x else (x if x else None)
        )
        
        # Step 3: Fetch transactions data
        transactions_query = "SELECT investorcode, schemename, foliono, trandate FROM MUTUALFUND_TRANSACTIONS"
        transactions_df = pd.read_sql(transactions_query, conn)
        
        # Preprocess transaction folio numbers similarly
        transactions_df['processed_folio'] = transactions_df['foliono'].apply(
            lambda x: x.split('/')[0] if x and '/' in x else (x if x else None)
        )
        
        # Initialize result dataframe
        results = []
        
        # Step 4: Process each order book entry
        for _, row in order_book_df.iterrows():
            # Create matching conditions
            condition1 = (
                (transactions_df['investorcode'] == row['investorcode']) &
                (transactions_df['schemename'] == row['schemename']) &
                (transactions_df['processed_folio'] == row['processed_folio']) &
                (pd.notna(row['processed_folio']))
            )
            
            condition2 = (
                (transactions_df['investorcode'] == row['investorcode']) &
                (transactions_df['schemename'] == row['schemename']) &
                (pd.isna(row['processed_folio']))
            )
            
            # Find matching transactions
            matching_trans = transactions_df[condition1 | condition2]
            
            # Filter transactions after registration date
            if not pd.isna(row['regdate']):
                matching_trans = matching_trans[
                    matching_trans['trandate'] >= row['regdate']
                ]
            
            # Find the earliest transaction date if any matches
            if not matching_trans.empty:
                first_trandate = matching_trans['trandate'].min()
                results.append({
                    'regno': row['regno'],
                    'investorcode': row['investorcode'],
                    'schemename': row['schemename'],
                    'foliono': row['foliono'],
                    'regdate': row['regdate'],
                    'first_trandate': first_trandate,
                    'has_matching_transaction': True
                })
            else:
                results.append({
                    'regno': row['regno'],
                    'investorcode': row['investorcode'],
                    'schemename': row['schemename'],
                    'foliono': row['foliono'],
                    'regdate': row['regdate'],
                    'first_trandate': None,
                    'has_matching_transaction': False
                })
        
        # Convert results to DataFrame
        result_df = pd.DataFrame(results)
        
        return result_df
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return pd.DataFrame()
    finally:
        if 'conn' in locals():
            conn.close()

# Example usage
if __name__ == "__main__":
    # Replace with your actual database credentials
    server = '192.168.100.55'
    database = 'Wealthone'
    username = 'aruhat'
    password = 'aruhat'
    
    result = process_mf_transactions(server, database, username, password)
    
    if not result.empty:
        print("Processing completed. Results:")
        # print(result.head())
        result.head()
        result.to_excel('last_result.xlsx', index = False)
        print("Excel File Created")
        
        # You can now work with the result DataFrame
        # For example, filter only records with matching transactions
        matched_records = result[result['has_matching_transaction']]
        print(f"\nFound {len(matched_records)} records with matching transactions.")
    else:
        print("No results returned from processing.")

C:\Users\ashish.ahirwar\AppData\Local\Temp\ipykernel_14100\4028665715.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  order_book_df = pd.read_sql(order_book_query, conn)
C:\Users\ashish.ahirwar\AppData\Local\Temp\ipykernel_14100\4028665715.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  transactions_df = pd.read_sql(transactions_query, conn)


Processing completed. Results:
Excel File Created

Found 0 records with matching transactions.


In [21]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import warnings

def process_mf_transactions(server, database, username, password):
    """
    Process MF order book and transactions data with accurate matching logic.
    
    Args:
        server (str): SQL server name
        database (str): Database name
        username (str): Database username
        password (str): Database password
    
    Returns:
        pd.DataFrame: Results with matching transactions
    """
    try:
        # Suppress SQLAlchemy warnings
        warnings.filterwarnings('ignore')
        
        # Create SQLAlchemy engine
        engine = create_engine(
            f"mssql+pyodbc://{username}:{password}@{server}/{database}?"
            "driver=ODBC+Driver+17+for+SQL+Server"
        )
        
        # Step 1: Fetch unique accepted orders
        order_book_query = """
        SELECT DISTINCT regno, investorcode, schemename, foliono, regdate
        FROM mf_order_book
        WHERE orderstatus = 'Accepted'
        """
        order_book_df = pd.read_sql(order_book_query, engine)
        
        if order_book_df.empty:
            print("No accepted orders found in mf_order_book")
            return pd.DataFrame()

        # Step 2: Preprocess folio numbers
        def process_folio(x):
            if pd.isna(x):
                return None
            x = str(x).strip()
            return x.split('/')[0] if '/' in x else x
        
        order_book_df['processed_folio'] = order_book_df['foliono'].apply(process_folio)
        
        # Step 3: Fetch transactions data
        transactions_query = """
        SELECT investorcode, schemename, foliono, trandate 
        FROM MUTUALFUND_TRANSACTIONS
        """
        transactions_df = pd.read_sql(transactions_query, engine)
        transactions_df['processed_folio'] = transactions_df['foliono'].apply(process_folio)
        
        # Convert dates to datetime if they're strings
        for df in [order_book_df, transactions_df]:
            if 'regdate' in df.columns and df['regdate'].dtype == object:
                df['regdate'] = pd.to_datetime(df['regdate'])
            if 'trandate' in df.columns and df['trandate'].dtype == object:
                df['trandate'] = pd.to_datetime(df['trandate'])

        # Initialize result list
        results = []
        
        # Step 4: Process each order with accurate matching
        for _, order in order_book_df.iterrows():
            # Base matching conditions
            base_match = (
                (transactions_df['investorcode'] == order['investorcode']) &
                (transactions_df['schemename'] == order['schemename'])
            )
            
            # Folio matching condition
            if pd.notna(order['processed_folio']):
                folio_match = (transactions_df['processed_folio'] == order['processed_folio'])
                matching_trans = transactions_df[base_match & folio_match]
            else:
                matching_trans = transactions_df[base_match]
            
            # Date filtering
            if pd.notna(order['regdate']):
                matching_trans = matching_trans[matching_trans['trandate'] >= order['regdate']]
            
            # Prepare result
            result = {
                'regno': order['regno'],
                'investorcode': order['investorcode'],
                'schemename': order['schemename'],
                'foliono': order['foliono'],
                'regdate': order['regdate'],
                'first_trandate': None,
                'has_matching_transaction': False,
                'matching_transaction_count': 0
            }
            
            if not matching_trans.empty:
                first_trandate = matching_trans['trandate'].min()
                result.update({
                    'first_trandate': first_trandate,
                    'has_matching_transaction': True,
                    'matching_transaction_count': len(matching_trans)
                })
            
            results.append(result)
        
        # Create final DataFrame
        result_df = pd.DataFrame(results)
        
        return result_df
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return pd.DataFrame()
    finally:
        if 'engine' in locals():
            engine.dispose()

if __name__ == "__main__":
    # Configuration
    server = '192.168.100.55'
    database = 'WEALTHONE'
    username = 'aruhat'
    password = 'aruhat'
    
    # Process data
    result = process_mf_transactions(server, database, username, password)
    
    if not result.empty:
        print("Processing completed successfully!")
        print(f"Total records processed: {len(result)}")
        print(f"Records with matches: {result['has_matching_transaction'].sum()}")
        
        # Save to Excel with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"mf_matching_results_{timestamp}.xlsx"
        result.to_excel(output_file, index=False)
        print(f"Results saved to: {output_file}")
        
        # Display sample of non-matching records for verification
        non_matches = result[~result['has_matching_transaction']]
        if not non_matches.empty:
            print("\nSample of non-matching records for verification:")
            print(non_matches[['regno', 'investorcode', 'schemename']].head())
    else:
        print("No results returned from processing.")

Processing completed successfully!
Total records processed: 1475
Records with matches: 0
Results saved to: mf_matching_results_20250414_124750.xlsx

Sample of non-matching records for verification:
       regno investorcode                                         schemename
0  100257205      RA10003  DSP INDIA T.I.G.E.R. FUND - REGULAR PLAN - IDC...
1  100258790    KUN105412    HDFC MID-CAP OPPORTUNITIES FUND - GROWTH OPTION
2  100401671       GK2003  MIRAE ASSET LARGE & MIDCAP FUND - REGULAR PLAN...
3  100994083      LS10008  NIPPON INDIA LARGE CAP FUND - GROWTH PLAN - GR...
4  100994194      LS10008    HDFC MANUFACTURING FUND - REGULAR PLAN - GROWTH


In [27]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import warnings

def process_mf_transactions(server, database, username, password):
    """
    Process MF order book and transactions data with precise matching logic.
    
    Args:
        server (str): SQL server name
        database (str): Database name
        username (str): Database username
        password (str): Database password
    
    Returns:
        pd.DataFrame: Results with matching transactions
    """
    try:
        # Suppress warnings
        warnings.filterwarnings('ignore')
        
        # Create SQLAlchemy engine
        engine = create_engine(
            f"mssql+pyodbc://{username}:{password}@{server}/{database}?"
            "driver=ODBC+Driver+17+for+SQL+Server"
        )
        
        # Step 1: Fetch accepted orders
        order_book_query = """
        SELECT DISTINCT regno, investorcode, schemename, foliono, regdate
        FROM mf_order_book
        WHERE orderstatus = 'Accepted'
        """
        order_book_df = pd.read_sql(order_book_query, engine)
        
        if order_book_df.empty:
            print("No accepted orders found in mf_order_book")
            return pd.DataFrame()

        # Step 2: Preprocess folio numbers
        def process_folio(x):
            if pd.isna(x):
                return None
            x = str(x).strip()
            return x.split('/')[0] if '/' in x else x
        
        order_book_df['processed_folio'] = order_book_df['foliono'].apply(process_folio)
        
        # Step 3: Fetch transactions data
        transactions_query = """
        SELECT investorcode, schemename, foliono, trandate 
        FROM MUTUALFUND_TRANSACTIONS
        """
        transactions_df = pd.read_sql(transactions_query, engine)
        transactions_df['processed_folio'] = transactions_df['foliono'].apply(process_folio)
        
        # Convert dates to datetime objects
        order_book_df['regdate'] = pd.to_datetime(order_book_df['regdate'])
        transactions_df['trandate'] = pd.to_datetime(transactions_df['trandate'])

        # Initialize result list
        results = []
        
        # Step 4: Process each order with precise matching
        for _, order in order_book_df.iterrows():
            # Base matching conditions
            investor_match = (transactions_df['investorcode'] == order['investorcode'])
            scheme_match = (transactions_df['schemename'].str.strip().str.upper() == 
                          order['schemename'].strip().upper())
            
            # Folio matching (case insensitive and with/without prefix)
            folio_condition = True  # Default to True if no folio
            if pd.notna(order['processed_folio']):
                folio_condition = (
                    transactions_df['processed_folio'].str.strip().str.upper() == 
                    order['processed_folio'].strip().upper()
                )
            
            # Combine all conditions
            matching_trans = transactions_df[
                investor_match & 
                scheme_match & 
                folio_condition
            ]
            
            # Filter transactions after registration date
            if pd.notna(order['regdate']):
                matching_trans = matching_trans[matching_trans['trandate'] >= order['regdate']]
            
            # Prepare result
            result = {
                'regno': order['regno'],
                'investorcode': order['investorcode'],
                'schemename': order['schemename'],
                'foliono': order['foliono'],
                'regdate': order['regdate'],
                'first_trandate': None,
                'has_matching_transaction': False,
                'matching_transaction_count': 0,
                'all_matching_dates': []
            }
            
            if not matching_trans.empty:
                # Get all matching dates sorted
                all_dates = sorted(matching_trans['trandate'].unique())
                first_trandate = all_dates[0]
                
                result.update({
                    'first_trandate': first_trandate,
                    'has_matching_transaction': True,
                    'matching_transaction_count': len(matching_trans),
                    'all_matching_dates': ", ".join([str(d.date()) for d in all_dates])
                })
            
            results.append(result)
        
        # Create final DataFrame
        result_df = pd.DataFrame(results)
        
        return result_df
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return pd.DataFrame()
    finally:
        if 'engine' in locals():
            engine.dispose()

if __name__ == "__main__":
    # Configuration
    server = '192.168.100.55'
    database = 'Wealthone'
    username = 'aruhat'
    password = 'aruhat'
    
    # Process data
    result = process_mf_transactions(server, database, username, password)
    
    if not result.empty:
        print("Processing completed successfully!")
        print(f"Total records processed: {len(result)}")
        print(f"Records with matches: {result['has_matching_transaction'].sum()}")
        
        # Save to Excel with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"mf_matching_results_{timestamp}.xlsx"
        
        # Format dates for better Excel display
        result['regdate'] = result['regdate'].dt.strftime('%Y-%m-%d')
        result['first_trandate'] = result['first_trandate'].dt.strftime('%Y-%m-%d')
        
        result.to_excel(output_file, index=False)
        print(f"Results saved to: {output_file}")
        
        # Display sample of matching records for verification
        matches = result[result['has_matching_transaction']]
        if not matches.empty:
            print("\nSample of matching records for verification:")
            print(matches[['regno', 'investorcode', 'schemename', 
                          'regdate', 'first_trandate']].head())
    else:
        print("No results returned from processing.")

An error occurred: 'Timestamp' object is not iterable
No results returned from processing.


In [31]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import warnings
import re

def normalize_scheme_name(name):
    """
    Normalize scheme names for flexible matching by:
    - Converting to lowercase
    - Removing extra spaces and punctuation
    - Standardizing common variations
    """
    if pd.isna(name):
        return ""
    
    # Convert to string and lowercase
    name = str(name).lower()
    
    # Remove special characters except spaces and hyphens
    name = re.sub(r'[^\w\s-]', '', name)
    
    # Standardize common variations
    replacements = {
        'regular plan': 'regular',
        'growth option': 'growth',
        'idcw reinvestment': 'idcw',
        'dividend reinvestment': 'idcw',
        'direct plan': 'direct'
    }
    
    for old, new in replacements.items():
        name = name.replace(old, new)
    
    # Remove extra spaces
    name = ' '.join(name.split())
    
    return name.strip()

def process_mf_transactions(server, database, username, password):
    """
    Process MF order book and transactions data with flexible scheme name matching.
    """
    try:
        warnings.filterwarnings('ignore')
        
        # Create SQLAlchemy engine
        engine = create_engine(
            f"mssql+pyodbc://{username}:{password}@{server}/{database}?"
            "driver=ODBC+Driver+17+for+SQL+Server"
        )
        
        # Step 1: Fetch accepted orders
        order_book_query = """
        SELECT DISTINCT regno, investorcode, schemename, foliono, regdate
        FROM mf_order_book
        WHERE orderstatus = 'Accepted'
        """
        order_book_df = pd.read_sql(order_book_query, engine)
        
        if order_book_df.empty:
            print("No accepted orders found in mf_order_book")
            return pd.DataFrame()

        # Step 2: Preprocess data
        def process_folio(x):
            if pd.isna(x):
                return None
            x = str(x).strip()
            return x.split('/')[0] if '/' in x else x
        
        # Process folio numbers and normalize scheme names
        order_book_df['processed_folio'] = order_book_df['foliono'].apply(process_folio)
        order_book_df['normalized_scheme'] = order_book_df['schemename'].apply(normalize_scheme_name)
        
        # Step 3: Fetch transactions data
        transactions_query = """
        SELECT investorcode, schemename, foliono, trandate 
        FROM MUTUALFUND_TRANSACTIONS
        """
        transactions_df = pd.read_sql(transactions_query, engine)
        transactions_df['processed_folio'] = transactions_df['foliono'].apply(process_folio)
        transactions_df['normalized_scheme'] = transactions_df['schemename'].apply(normalize_scheme_name)
        
        # Convert dates to datetime objects
        order_book_df['regdate'] = pd.to_datetime(order_book_df['regdate'])
        transactions_df['trandate'] = pd.to_datetime(transactions_df['trandate'])

        # Initialize result list
        results = []
        
        # Step 4: Process each order with flexible matching
        for _, order in order_book_df.iterrows():
            # Base matching conditions
            investor_match = (transactions_df['investorcode'] == order['investorcode'])
            scheme_match = (transactions_df['normalized_scheme'] == order['normalized_scheme'])
            
            # Folio matching
            folio_condition = True  # Default to True if no folio
            if pd.notna(order['processed_folio']):
                folio_condition = (
                    transactions_df['processed_folio'].astype(str).str.strip().str.lower() == 
                    order['processed_folio'].strip().lower()
                )
            
            # Combine all conditions
            matching_trans = transactions_df[
                investor_match & 
                scheme_match & 
                folio_condition
            ]
            
            # Filter transactions after registration date
            if pd.notna(order['regdate']):
                matching_trans = matching_trans[matching_trans['trandate'] >= order['regdate']]
            
            # Prepare result
            result = {
                'regno': order['regno'],
                'investorcode': order['investorcode'],
                'schemename': order['schemename'],
                'transaction_schemename': ', '.join(transactions_df[matching_trans.index]['schemename'].unique()),
                'foliono': order['foliono'],
                'regdate': order['regdate'],
                'first_trandate': None,
                'has_matching_transaction': False,
                'matching_transaction_count': 0,
                'all_matching_dates': []
            }
            
            if not matching_trans.empty:
                # Get all matching dates sorted
                all_dates = sorted(matching_trans['trandate'].unique())
                first_trandate = all_dates[0]
                
                result.update({
                    'first_trandate': first_trandate,
                    'has_matching_transaction': True,
                    'matching_transaction_count': len(matching_trans),
                    'all_matching_dates': ", ".join([str(d.date()) for d in all_dates])
                })
            
            results.append(result)
        
        # Create final DataFrame
        result_df = pd.DataFrame(results)
        
        return result_df
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return pd.DataFrame()
    finally:
        if 'engine' in locals():
            engine.dispose()

if __name__ == "__main__":
    # Configuration
    server = '192.168.100.55'
    database = 'Wealthone'
    username = 'aruhat'
    password = 'aruhat'
    
    # Process data
    result = process_mf_transactions(server, database, username, password)
    
    if not result.empty:
        print("Processing completed successfully!")
        print(f"Total records processed: {len(result)}")
        print(f"Records with matches: {result['has_matching_transaction'].sum()}")
        
        # Save to Excel with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"mf_matching_results_{timestamp}.xlsx"
        
        # Format dates for better Excel display
        date_cols = ['regdate', 'first_trandate']
        for col in date_cols:
            if col in result.columns:
                result[col] = result[col].apply(lambda x: x.strftime('%Y-%m-%d') if not pd.isna(x) else '')
        
        result.to_excel(output_file, index=False)
        print(f"Results saved to: {output_file}")
        
        # Display sample records for verification
        print("\nSample records for verification:")
        sample_cols = ['regno', 'investorcode', 'schemename', 'transaction_schemename', 
                      'regdate', 'first_trandate', 'matching_transaction_count']
        print(result[sample_cols].head(10))
    else:
        print("No results returned from processing.")

An error occurred: "None of [Index([21981, 26146, 29936, 151282, 153476, 155001], dtype='int64')] are in the [columns]"
No results returned from processing.


In [59]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import warnings
import re
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='mf_matching.log'
)
logger = logging.getLogger(__name__)

def normalize_text(text):
    """Normalize text for comparison by removing special chars and standardizing case"""
    if pd.isna(text):
        return ""
    text = str(text).lower().strip()
    text = re.sub(r'[^\w\s-]', '', text)  # Remove special chars except hyphens
    text = re.sub(r'\s+', ' ', text)  # Collapse multiple spaces
    return text

def normalize_scheme_name(name):
    """Special normalization for mutual fund scheme names"""
    name = normalize_text(name)
    
    # Standardize common variations
    replacements = {
        'regular plan': 'regular',
        'growth option': 'growth',
        'idcw reinvestment': 'idcw',
        'dividend reinvestment': 'idcw',
        'direct plan': 'direct',
        'fund -': 'fund',
        'plan -': 'plan'
    }
    
    for old, new in replacements.items():
        name = name.replace(old, new)
    
    return name.strip()

def process_folio(folio):
    """Process folio numbers consistently"""
    if pd.isna(folio):
        return None
    folio = str(folio).strip()
    return folio.split('/')[0] if '/' in folio else folio

def get_db_connection(server, database, username, password):
    """Create and return a SQLAlchemy engine with error handling"""
    try:
        conn_str = (
            f"mssql+pyodbc://{username}:{password}@{server}/{database}?"
            f"driver=ODBC+Driver+17+for+SQL+Server"
        )
        return create_engine(conn_str)
    except Exception as e:
        logger.error(f"Database connection failed: {str(e)}")
        raise

def validate_dataframes(order_df, trans_df):
    """Validate the structure of input DataFrames"""
    required_order_cols = {'regno', 'investorcode', 'schemename', 'foliono', 'regdate'}
    required_trans_cols = {'investorcode', 'schemename', 'foliono', 'trandate'}
    
    if not required_order_cols.issubset(order_df.columns):
        missing = required_order_cols - set(order_df.columns)
        raise ValueError(f"Order book missing columns: {missing}")
    
    if not required_trans_cols.issubset(trans_df.columns):
        missing = required_trans_cols - set(trans_df.columns)
        raise ValueError(f"Transactions missing columns: {missing}")
    
    logger.info("Data validation passed")

def process_mf_transactions(server, database, username, password):
    """Main processing function with comprehensive error handling"""
    try:
        warnings.filterwarnings('ignore')
        logger.info("Starting MF transaction processing")
        
        # Establish database connection
        engine = get_db_connection(server, database, username, password)
        
        # Step 1: Fetch accepted orders
        logger.info("Fetching order book data")
        order_book_query = """
        SELECT DISTINCT regno, investorcode, amcid, foliono, regdate
        FROM MF_LIVE
        WHERE orderstatus = 'Accepted'
        """
        order_book_df = pd.read_sql(order_book_query, engine)
        
        if order_book_df.empty:
            logger.warning("No accepted orders found in mf_order_book")
            return pd.DataFrame()

        # Step 2: Fetch transactions data
        logger.info("Fetching transactions data")
        transactions_query = """
        SELECT investorcode, amcid, foliono, trandate 
        FROM MUTUALFUND_TRANSACTIONS
        """
        transactions_df = pd.read_sql(transactions_query, engine)
        
        # Validate data structure
        validate_dataframes(order_book_df, transactions_df)
        
        # Step 3: Preprocess data
        logger.info("Preprocessing data")
        
        # Process folio numbers
        order_book_df['processed_folio'] = order_book_df['foliono'].apply(process_folio)
        transactions_df['processed_folio'] = transactions_df['foliono'].apply(process_folio)
        
        # Normalize scheme names
        order_book_df['normalized_scheme'] = order_book_df['amcid'].apply(normalize_scheme_name)
        transactions_df['normalized_scheme'] = transactions_df['amcid'].apply(normalize_scheme_name)
        
        # Convert dates to datetime
        order_book_df['regdate'] = pd.to_datetime(order_book_df['regdate'])
        transactions_df['trandate'] = pd.to_datetime(transactions_df['trandate'])
        
        # Initialize results
        results = []
        match_count = 0
        
        # Step 4: Process each order
        logger.info(f"Processing {len(order_book_df)} orders")
        for idx, order in order_book_df.iterrows():
            try:
                # Create matching conditions
                investor_cond = (transactions_df['investorcode'] == order['investorcode'])
                scheme_cond = (transactions_df['normalized_scheme'] == order['normalized_scheme'])
                
                # Folio condition (skip if folio is null/empty)
                folio_cond = True
                if pd.notna(order['processed_folio']) and order['processed_folio'] != '':
                    folio_cond = (
                        transactions_df['processed_folio'].astype(str) == 
                        order['processed_folio'].astype(str)
                    )
                
                # Combine conditions
                matching_trans = transactions_df[investor_cond & scheme_cond & folio_cond]
                
                # Filter by date
                if pd.notna(order['regdate']):
                    matching_trans = matching_trans[matching_trans['trandate'] >= order['regdate']]
                
                # Prepare result
                result = {
                    'regno': order['regno'],
                    'investorcode': order['investorcode'],
                    'order_schemename': order['amcid'],
                    'order_folio': order['foliono'],
                    'regdate': order['regdate'],
                    'first_trandate': None,
                    'has_matching_transaction': False,
                    'match_count': 0,
                    'matching_transaction_schemes': None,
                    'matching_dates': None
                }
                
                if not matching_trans.empty:
                    match_count += 1
                    first_date = matching_trans['trandate'].min()
                    unique_schemes = matching_trans['amcid'].unique()
                    
                    result.update({
                        'first_trandate': first_date,
                        'has_matching_transaction': True,
                        'match_count': len(matching_trans),
                        'matching_transaction_schemes': ', '.join(unique_schemes),
                        'matching_dates': ', '.join(
                            matching_trans['trandate'].dt.strftime('%Y-%m-%d').sort_values().unique()
                        )
                    })
                
                results.append(result)
                
            except Exception as e:
                logger.error(f"Error processing order {order['regno']}: {str(e)}")
                continue
        
        logger.info(f"Processing complete. Found matches for {match_count}/{len(order_book_df)} orders")
        
        # Create final DataFrame
        result_df = pd.DataFrame(results)
        
        # Convert dates to strings for Excel output
        date_cols = ['regdate', 'first_trandate']
        for col in date_cols:
            if col in result_df.columns:
                result_df[col] = result_df[col].dt.strftime('%Y-%m-%d')
        
        return result_df
        
    except Exception as e:
        logger.error(f"Processing failed: {str(e)}", exc_info=True)
        return pd.DataFrame()
    finally:
        if 'engine' in locals():
            engine.dispose()
            logger.info("Database connection closed")

if __name__ == "__main__":
    try:
        # Configuration
        config = {
            'server' : '192.168.100.55',
            'database' : 'Wealthone',
            'username' : 'aruhat',
            'password' : 'aruhat'
        }
        
        # Process data
        logger.info("Starting main execution")
        result = process_mf_transactions(**config)
        
        if not result.empty:
            # Save results
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file = f"mf_matching_results_{timestamp}.xlsx"
            
            # Reorder columns for better readability
            cols = [
                'regno', 'investorcode', 'order_schemename', 
                'matching_transaction_schemes', 'order_folio',
                'regdate', 'first_trandate', 'match_count',
                'has_matching_transaction', 'matching_dates'
            ]
            cols = [c for c in cols if c in result.columns]
            
            result[cols].to_excel(output_file, index=False)
            logger.info(f"Results saved to {output_file}")
            
            # Print summary
            print(f"\nProcessing Summary:")
            print(f"Total orders processed: {len(result)}")
            print(f"Orders with matches: {result['has_matching_transaction'].sum()}")
            print(f"Output file: {output_file}")
            
            # Show sample non-matches for verification
            non_matches = result[~result['has_matching_transaction']]
            if not non_matches.empty:
                print("\nSample non-matching orders for verification:")
                print(non_matches[['regno', 'investorcode', 'order_schemename']].head(5))
        else:
            logger.warning("No results returned")
            print("Processing completed but no results returned. Check log file for details.")
            
    except Exception as e:
        logger.error(f"Fatal error in main execution: {str(e)}", exc_info=True)
        print(f"An error occurred. Please check the log file for details.")

Processing completed but no results returned. Check log file for details.


In [41]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import warnings
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='mf_amcid_matching.log'
)
logger = logging.getLogger(__name__)

def process_folio(folio):
    """Process folio numbers consistently"""
    if pd.isna(folio):
        return None
    folio = str(folio).strip()
    return folio.split('/')[0] if '/' in folio else folio

def get_db_connection(server, database, username, password):
    """Create and return a SQLAlchemy engine"""
    try:
        conn_str = (
            f"mssql+pyodbc://{username}:{password}@{server}/{database}?"
            f"driver=ODBC+Driver+17+for+SQL+Server"
        )
        return create_engine(conn_str)
    except Exception as e:
        logger.error(f"Database connection failed: {str(e)}")
        raise

def validate_dataframes(order_df, trans_df):
    """Validate the structure of input DataFrames"""
    required_order_cols = {'regno', 'investorcode', 'amcid', 'foliono', 'regdate'}
    required_trans_cols = {'investorcode', 'amcid', 'foliono', 'trandate'}
    
    if not required_order_cols.issubset(order_df.columns):
        missing = required_order_cols - set(order_df.columns)
        raise ValueError(f"Order book missing columns: {missing}")
    
    if not required_trans_cols.issubset(trans_df.columns):
        missing = required_trans_cols - set(trans_df.columns)
        raise ValueError(f"Transactions missing columns: {missing}")
    
    logger.info("Data validation passed")

def process_mf_transactions(server, database, username, password):
    """Main processing function using AMCID for exact matching"""
    try:
        warnings.filterwarnings('ignore')
        logger.info("Starting MF transaction processing with AMCID matching")
        
        # Establish database connection
        engine = get_db_connection(server, database, username, password)
        
        # Step 1: Fetch accepted orders with AMCID
        logger.info("Fetching order book data with AMCID")
        order_book_query = """
        SELECT DISTINCT o.regno, o.investorcode, o.foliono, o.regdate, l.amcid
        FROM mf_order_book o
        JOIN mf_live l ON o.schemename = l.schemename  -- Or appropriate join condition
        WHERE o.orderstatus = 'Accepted'
        """
        order_book_df = pd.read_sql(order_book_query, engine)
        
        if order_book_df.empty:
            logger.warning("No accepted orders found in mf_order_book")
            return pd.DataFrame()

        # Step 2: Fetch transactions data with AMCID
        logger.info("Fetching transactions data with AMCID")
        transactions_query = """
        SELECT t.investorcode, t.foliono, t.trandate, l.amcid
        FROM MUTUALFUND_TRANSACTIONS t
        JOIN mf_live l ON t.schemename = l.schemename  -- Or appropriate join condition
        """
        transactions_df = pd.read_sql(transactions_query, engine)
        
        # Validate data structure
        validate_dataframes(order_book_df, transactions_df)
        
        # Step 3: Preprocess data
        logger.info("Preprocessing data")
        
        # Process folio numbers
        order_book_df['processed_folio'] = order_book_df['foliono'].apply(process_folio)
        transactions_df['processed_folio'] = transactions_df['foliono'].apply(process_folio)
        
        # Convert dates to datetime
        order_book_df['regdate'] = pd.to_datetime(order_book_df['regdate'])
        transactions_df['trandate'] = pd.to_datetime(transactions_df['trandate'])
        
        # Initialize results
        results = []
        match_count = 0
        
        # Step 4: Process each order using AMCID
        logger.info(f"Processing {len(order_book_df)} orders")
        for idx, order in order_book_df.iterrows():
            try:
                # Create matching conditions
                investor_cond = (transactions_df['investorcode'] == order['investorcode'])
                amcid_cond = (transactions_df['amcid'] == order['amcid'])
                
                # Folio condition (skip if folio is null/empty)
                folio_cond = True
                if pd.notna(order['processed_folio']) and order['processed_folio'] != '':
                    folio_cond = (
                        transactions_df['processed_folio'].astype(str) == 
                        order['processed_folio'].astype(str)
                    )
                
                # Combine conditions
                matching_trans = transactions_df[investor_cond & amcid_cond & folio_cond]
                
                # Filter by date
                if pd.notna(order['regdate']):
                    matching_trans = matching_trans[matching_trans['trandate'] >= order['regdate']]
                
                # Prepare result
                result = {
                    'regno': order['regno'],
                    'investorcode': order['investorcode'],
                    'amcid': order['amcid'],
                    'order_folio': order['foliono'],
                    'regdate': order['regdate'],
                    'first_trandate': None,
                    'has_matching_transaction': False,
                    'match_count': 0,
                    'matching_dates': None
                }
                
                if not matching_trans.empty:
                    match_count += 1
                    first_date = matching_trans['trandate'].min()
                    
                    result.update({
                        'first_trandate': first_date,
                        'has_matching_transaction': True,
                        'match_count': len(matching_trans),
                        'matching_dates': ', '.join(
                            matching_trans['trandate'].dt.strftime('%Y-%m-%d').sort_values().unique()
                        )
                    })
                
                results.append(result)
                
            except Exception as e:
                logger.error(f"Error processing order {order['regno']}: {str(e)}")
                continue
        
        logger.info(f"Processing complete. Found matches for {match_count}/{len(order_book_df)} orders")
        
        # Create final DataFrame
        result_df = pd.DataFrame(results)
        
        # Convert dates to strings for Excel output
        date_cols = ['regdate', 'first_trandate']
        for col in date_cols:
            if col in result_df.columns:
                result_df[col] = result_df[col].dt.strftime('%Y-%m-%d')
        
        return result_df
        
    except Exception as e:
        logger.error(f"Processing failed: {str(e)}", exc_info=True)
        return pd.DataFrame()
    finally:
        if 'engine' in locals():
            engine.dispose()
            logger.info("Database connection closed")

if __name__ == "__main__":
    try:
        # Configuration
        config = {
            'server': '192.168.100.55',
            'database': 'Wealthone',
            'username': 'aruhat',
            'password': 'aruhat'
        }
        
        # Process data
        logger.info("Starting main execution with AMCID matching")
        result = process_mf_transactions(**config)
        
        if not result.empty:
            # Save results
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file = f"mf_amcid_matching_results_{timestamp}.xlsx"
            
            # Reorder columns for better readability
            cols = [
                'regno', 'investorcode', 'amcid', 'order_folio',
                'regdate', 'first_trandate', 'match_count',
                'has_matching_transaction', 'matching_dates'
            ]
            cols = [c for c in cols if c in result.columns]
            
            result[cols].to_excel(output_file, index=False)
            logger.info(f"Results saved to {output_file}")
            
            # Print summary
            print(f"\nProcessing Summary:")
            print(f"Total orders processed: {len(result)}")
            print(f"Orders with matches: {result['has_matching_transaction'].sum()}")
            print(f"Output file: {output_file}")
            
            # Show sample non-matches for verification
            non_matches = result[~result['has_matching_transaction']]
            if not non_matches.empty:
                print("\nSample non-matching orders for verification:")
                print(non_matches[['regno', 'investorcode', 'amcid']].head(5))
        else:
            logger.warning("No results returned")
            print("Processing completed but no results returned. Check log file for details.")
            
    except Exception as e:
        logger.error(f"Fatal error in main execution: {str(e)}", exc_info=True)
        print(f"An error occurred. Please check the log file for details.")

Processing completed but no results returned. Check log file for details.


In [51]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import warnings
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='mf_amcid_matching.log'
)
logger = logging.getLogger(__name__)

def process_folio(folio):
    """Process folio numbers consistently"""
    if pd.isna(folio):
        return None
    folio = str(folio).strip()
    return folio.split('/')[0] if '/' in folio else folio

def get_db_connection(server, database, username, password):
    """Create and return a SQLAlchemy engine"""
    try:
        conn_str = (
            f"mssql+pyodbc://{username}:{password}@{server}/{database}?"
            f"driver=ODBC+Driver+17+for+SQL+Server"
        )
        return create_engine(conn_str)
    except Exception as e:
        logger.error(f"Database connection failed: {str(e)}")
        raise

def validate_dataframes(order_df, trans_df):
    """Validate the structure of input DataFrames"""
    required_order_cols = {'regno', 'investorcode', 'amcid', 'foliono', 'regdate'}
    required_trans_cols = {'investorcode', 'amcid', 'foliono', 'trandate'}
    
    if not required_order_cols.issubset(order_df.columns):
        missing = required_order_cols - set(order_df.columns)
        raise ValueError(f"Order book missing columns: {missing}")
    
    if not required_trans_cols.issubset(trans_df.columns):
        missing = required_trans_cols - set(trans_df.columns)
        raise ValueError(f"Transactions missing columns: {missing}")
    
    logger.info("Data validation passed")

def process_mf_transactions(server, database, username, password):
    """Main processing function using only AMCID for scheme matching"""
    try:
        warnings.filterwarnings('ignore')
        logger.info("Starting transaction processing with AMCID matching")
        
        # Establish database connection
        engine = get_db_connection(server, database, username, password)
        
        # Step 1: Fetch accepted orders (from your merged table)
        logger.info("Fetching order data with AMCID")
        order_query = """
        SELECT DISTINCT regno, investorcode, amcid, foliono, regdate
        FROM mf_live  
        WHERE orderstatus = 'Accepted'
        """
        order_df = pd.read_sql(order_query, engine)
        
        if order_df.empty:
            logger.warning("No accepted orders found")
            return pd.DataFrame()

        # Step 2: Fetch transactions data with AMCID
        logger.info("Fetching transactions data with AMCID")
        transactions_query = """
        SELECT investorcode, amcid, foliono, trandate
        FROM MUTUALFUND_TRANSACTIONS
        """
        trans_df = pd.read_sql(transactions_query, engine)
        
        # Validate data structure
        validate_dataframes(order_df, trans_df)
        
        # Step 3: Preprocess data
        logger.info("Preprocessing data")
        
        # Process folio numbers
        order_df['processed_folio'] = order_df['foliono'].apply(process_folio)
        trans_df['processed_folio'] = trans_df['foliono'].apply(process_folio)
        
        # Convert dates to datetime
        order_df['regdate'] = pd.to_datetime(order_df['regdate'])
        trans_df['trandate'] = pd.to_datetime(trans_df['trandate'])
        
        # Initialize results
        results = []
        match_count = 0
        
        # Step 4: Process each order using AMCID
        logger.info(f"Processing {len(order_df)} orders")
        for idx, order in order_df.iterrows():
            try:
                # Create matching conditions
                conditions = [
                    trans_df['investorcode'] == order['investorcode'],
                    trans_df['amcid'] == order['amcid']
                ]
                
                # Add folio condition if exists
                if pd.notna(order['processed_folio']) and order['processed_folio'] != '':
                    conditions.append(
                        trans_df['processed_folio'] == order['processed_folio']
                    )
                
                # Combine conditions
                matching_trans = trans_df[pd.concat(conditions, axis=1).all(axis=1)]
                
                # Filter by registration date
                if pd.notna(order['regdate']):
                    matching_trans = matching_trans[matching_trans['trandate'] >= order['regdate']]
                
                # Prepare result
                result = {
                    'regno': order['regno'],
                    'investorcode': order['investorcode'],
                    'amcid': order['amcid'],
                    'foliono': order['foliono'],
                    'regdate': order['regdate'],
                    'first_trandate': None,
                    'has_match': False,
                    'match_count': 0,
                    'matching_dates': None
                }
                
                if not matching_trans.empty:
                    match_count += 1
                    first_date = matching_trans['trandate'].min()
                    
                    result.update({
                        'first_trandate': first_date,
                        'has_match': True,
                        'match_count': len(matching_trans),
                        'matching_dates': ', '.join(
                            matching_trans['trandate'].dt.strftime('%Y-%m-%d').sort_values().unique()
                        )
                    })
                
                results.append(result)
                
            except Exception as e:
                logger.error(f"Error processing order {order['regno']}: {str(e)}")
                continue
        
        logger.info(f"Processing complete. Matches found: {match_count}/{len(order_df)}")
        
        # Create final DataFrame
        result_df = pd.DataFrame(results)
        
        # Convert dates to strings for Excel output
        date_cols = ['regdate', 'first_trandate']
        for col in date_cols:
            if col in result_df.columns:
                result_df[col] = result_df[col].dt.strftime('%Y-%m-%d')
        
        return result_df
        
    except Exception as e:
        logger.error(f"Processing failed: {str(e)}", exc_info=True)
        return pd.DataFrame()
    finally:
        if 'engine' in locals():
            engine.dispose()
            logger.info("Database connection closed")

if __name__ == "__main__":
    try:
        # Configuration
        config = {
            'server': '192.168.100.55',
            'database': 'Wealthone',
            'username': 'aruhat',
            'password': 'aruhat'
        }
        
        # Process data
        logger.info("Starting execution")
        result = process_mf_transactions(**config)
        
        if not result.empty:
            # Save results
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file = f"amcid_matching_results_{timestamp}.xlsx"
            
            # Select output columns
            output_cols = [
                'regno', 'investorcode', 'amcid', 'foliono',
                'regdate', 'first_trandate', 'match_count',
                'has_match', 'matching_dates'
            ]
            result[output_cols].to_excel(output_file, index=False)
            
            # Print summary
            print(f"\nProcessing Summary:")
            print(f"Total orders: {len(result)}")
            print(f"Matched orders: {result['has_match'].sum()}")
            print(f"Results saved to: {output_file}")
            
            # Show sample non-matches for verification
            non_matches = result[~result['has_match']]
            if not non_matches.empty:
                print("\nSample non-matching orders:")
                print(non_matches[['regno', 'investorcode', 'amcid']].head())
        else:
            print("Processing completed with no results. Check log for details.")
            
    except Exception as e:
        logger.error(f"Fatal error: {str(e)}", exc_info=True)
        print("An error occurred. Check log file for details.")

Processing completed with no results. Check log for details.


In [53]:
import pandas as pd
from sqlalchemy import create_engine, exc
from datetime import datetime
import logging
import traceback

# Enhanced logging configuration
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='mf_amcid_matching_debug.log',
    filemode='w'
)
logger = logging.getLogger(__name__)

def get_db_connection(server, database, username, password):
    """Enhanced connection function with timeout and validation"""
    try:
        conn_str = (
            f"mssql+pyodbc://{username}:{password}@{server}/{database}?"
            f"driver=ODBC+Driver+17+for+SQL+Server&"
            f"timeout=30&login_timeout=10"
        )
        engine = create_engine(conn_str)
        
        # Test connection immediately
        with engine.connect() as test_conn:
            test_conn.execute("SELECT 1")
            
        logger.info("Database connection established successfully")
        return engine
        
    except exc.SQLAlchemyError as e:
        logger.error(f"Database connection failed: {str(e)}")
        logger.debug(traceback.format_exc())
        raise
    except Exception as e:
        logger.error(f"Unexpected connection error: {str(e)}")
        raise

def validate_and_preprocess_data(order_df, trans_df):
    """Enhanced data validation with detailed logging"""
    logger.info("Starting data validation")
    
    # Check if DataFrames are empty
    if order_df.empty:
        logger.error("Order DataFrame is empty")
        raise ValueError("No order data found")
    if trans_df.empty:
        logger.error("Transactions DataFrame is empty")
        raise ValueError("No transaction data found")
    
    # Required columns check
    required_order_cols = {'regno', 'investorcode', 'amcid', 'folio', 'regdate'}
    required_trans_cols = {'investorcode', 'amcid', 'folio', 'trandate'}
    
    missing_order = required_order_cols - set(order_df.columns)
    missing_trans = required_trans_cols - set(trans_df.columns)
    
    if missing_order:
        logger.error(f"Missing columns in order data: {missing_order}")
    if missing_trans:
        logger.error(f"Missing columns in transaction data: {missing_trans}")
    if missing_order or missing_trans:
        raise ValueError("Missing required columns")
    
    # Data type conversion with error handling
    try:
        order_df['regdate'] = pd.to_datetime(order_df['regdate'], errors='coerce')
        trans_df['trandate'] = pd.to_datetime(trans_df['trandate'], errors='coerce')
    except Exception as e:
        logger.error(f"Date conversion failed: {str(e)}")
        logger.debug(traceback.format_exc())
        raise
    
    # Check for null AMCIDs
    if order_df['amcid'].isnull().any():
        logger.warning(f"Found {order_df['amcid'].isnull().sum()} orders with null AMCID")
    if trans_df['amcid'].isnull().any():
        logger.warning(f"Found {trans_df['amcid'].isnull().sum()} transactions with null AMCID")
    
    logger.info("Data validation completed successfully")
    return order_df, trans_df

def process_mf_transactions(server, database, username, password):
    """Main function with comprehensive debugging"""
    try:
        logger.info("=== Starting Processing ===")
        
        # 1. Establish connection
        engine = get_db_connection(server, database, username, password)
        
        # 2. Fetch order data with debug info
        logger.debug("Executing order query")
        order_query = """
        SELECT distinct regno, investorcode, amcid, foliono, regdate
        FROM MF_LIVE
        WHERE orderstatus = 'Accepted'
        """
        try:
            order_df = pd.read_sql(order_query, engine)
            logger.info(f"Retrieved {len(order_df)} orders")
            logger.debug(f"Order data sample:\n{order_df.head(2)}")
        except Exception as e:
            logger.error(f"Order query failed: {str(e)}")
            raise
        
        # 3. Fetch transaction data with debug info
        logger.debug("Executing transaction query")
        trans_query = """
        SELECT  investorcode, amcid, foliono, trandate
        FROM MUTUALFUND_TRANSACTIONS
        WHERE trandate >= DATEADD(month, -6, GETDATE())
        """
        try:
            trans_df = pd.read_sql(trans_query, engine)
            logger.info(f"Retrieved {len(trans_df)} transactions")
            logger.debug(f"Transaction data sample:\n{trans_df.head(2)}")
        except Exception as e:
            logger.error(f"Transaction query failed: {str(e)}")
            raise
        
        # 4. Validate and preprocess
        order_df, trans_df = validate_and_preprocess_data(order_df, trans_df)
        
        # 5. Processing with progress tracking
        results = []
        match_count = 0
        total_orders = len(order_df)
        
        logger.info(f"Processing {total_orders} orders against {len(trans_df)} transactions")
        
        for idx, order in order_df.iterrows():
            try:
                # Debug every 100th record
                if idx % 100 == 0:
                    logger.debug(f"Processing order {idx}/{total_orders} - RegNo: {order['regno']}")
                
                # Matching logic
                match_cond = (
                    (trans_df['investorcode'] == order['investorcode']) &
                    (trans_df['amcid'] == order['amcid'])
                )
                
                if pd.notna(order['foliono']):
                    match_cond &= (trans_df['foliono'] == order['foliono'])
                
                if pd.notna(order['regdate']):
                    match_cond &= (trans_df['trandate'] >= order['regdate'])
                
                matches = trans_df[match_cond]
                
                # Prepare result
                result = {
                    'regno': order['regno'],
                    'investorcode': order['investorcode'],
                    'amcid': order['amcid'],
                    'foliono': order['foliono'],
                    'regdate': order['regdate'],
                    'has_match': False,
                    'match_count': 0,
                    'first_match_date': None,
                    'last_match_date': None
                }
                
                if not matches.empty:
                    match_count += 1
                    result.update({
                        'has_match': True,
                        'match_count': len(matches),
                        'first_match_date': matches['trandate'].min(),
                        'last_match_date': matches['trandate'].max()
                    })
                
                results.append(result)
                
            except Exception as e:
                logger.error(f"Error processing order {order['regno']}: {str(e)}")
                continue
        
        # 6. Create final output
        result_df = pd.DataFrame(results)
        
        if result_df.empty:
            logger.warning("No results generated - empty DataFrame")
        else:
            logger.info(f"Processing complete. Matches found: {match_count}/{total_orders}")
            logger.debug(f"Result sample:\n{result_df.head()}")
        
        return result_df
        
    except Exception as e:
        logger.error(f"Fatal processing error: {str(e)}")
        logger.debug(traceback.format_exc())
        return None
    finally:
        if 'engine' in locals():
            engine.dispose()
            logger.info("Database connection closed")
        logger.info("=== Processing Completed ===")

if __name__ == "__main__":
    try:
        # Configuration - replace with your actual credentials
        config = {
            'server': '192.168.100.55',
            'database': 'Wealthone',
            'username': 'aruhat',
            'password': 'aruhat'
        }
        
        # Execute processing
        logger.info("Starting main execution")
        result = process_mf_transactions(**config)
        
        if result is not None and not result.empty:
            # Save results with timestamp
            output_file = f"amcid_results_{datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"
            result.to_excel(output_file, index=False)
            
            # Print summary
            print("\nProcessing Summary:")
            print(f"Orders Processed: {len(result)}")
            print(f"Orders Matched: {result['has_match'].sum()}")
            print(f"Results saved to: {output_file}")
            print(f"Detailed log: mf_amcid_matching_debug.log")
            
            # Show sample of non-matches for investigation
            non_matches = result[~result['has_match']]
            if not non_matches.empty:
                print("\nSample Non-Matched Orders:")
                print(non_matches[['regno', 'investorcode', 'amcid']].head())
        else:
            print("Processing completed with no results. Please check the log file for details.")
            
    except Exception as e:
        print(f"Fatal error occurred. Check log file for details: {str(e)}")

Processing completed with no results. Please check the log file for details.


In [69]:
import pandas as pd
from sqlalchemy import create_engine, exc
from datetime import datetime, timedelta
import logging
import traceback

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='mf_matching_debug.log',
    filemode='w'
)
logger = logging.getLogger(__name__)

def get_db_connection(server, database, username, password):
    """Create database connection with error handling"""
    try:
        conn_str = (
            f"mssql+pyodbc://{username}:{password}@{server}/{database}?"
            f"driver=ODBC+Driver+17+for+SQL+Server&"
            # f"timeout=30&login_timeout=10"
        )
        engine = create_engine(conn_str)
        # Test connection
        with engine.connect() as conn:
            conn.execute("SELECT 1")
        logger.info("Database connection successful")
        return engine
    except exc.SQLAlchemyError as e:
        logger.error(f"Connection failed: {str(e)}")
        raise

def validate_data(order_df, trans_df):
    """Validate data structure and content"""
    logger.info("Validating data")
    
    # Check required columns
    required_order = ['regno', 'investorcode', 'amcid', 'foliono', 'regdate']
    required_trans = ['investorcode', 'amcid', 'foliono', 'trandate']
    
    missing_order = [col for col in required_order if col not in order_df.columns]
    missing_trans = [col for col in required_trans if col not in trans_df.columns]
    
    if missing_order:
        logger.error(f"Missing columns in orders: {missing_order}")
    if missing_trans:
        logger.error(f"Missing columns in transactions: {missing_trans}")
    if missing_order or missing_trans:
        raise ValueError("Missing required columns")
    
    # Check for null AMCIDs
    if order_df['amcid'].isnull().any():
        logger.warning(f"{order_df['amcid'].isnull().sum()} orders have null AMCID")
    if trans_df['amcid'].isnull().any():
        logger.warning(f"{trans_df['amcid'].isnull().sum()} transactions have null AMCID")
    
    # Convert dates
    try:
        order_df['regdate'] = pd.to_datetime(order_df['regdate'], errors='coerce')
        trans_df['trandate'] = pd.to_datetime(trans_df['trandate'], errors='coerce')
    except Exception as e:
        logger.error(f"Date conversion failed: {str(e)}")
        raise
    
    logger.info("Data validation complete")
    return order_df, trans_df

def process_folio(folio):
    """Process folio numbers consistently"""
    if pd.isna(folio):
        return None
    folio = str(folio).strip()
    return folio.split('/')[0] if '/' in folio else folio

def process_matches(server, database, username, password):
    """Main processing function with correct column names"""
    try:
        logger.info("Starting MF matching process")
        
        # 1. Connect to database
        engine = get_db_connection(server, database, username, password)
        
        # 2. Fetch order data from mf_live
        order_query = """
        SELECT regno, investorcode, amcid, foliono, CAST(LEFT(Registrationdate, CHARINDEX('T', Registrationdate) - 1) AS DATE)  regdate
        FROM mf_live
        WHERE orderstatus = 'Accepted'
        """
        logger.debug("Fetching order data")
        order_df = pd.read_sql(order_query, engine)
        logger.info(f"Found {len(order_df)} accepted orders")
        
        if order_df.empty:
            logger.error("No accepted orders found")
            return pd.DataFrame()
        
        # 3. Fetch transaction data
        trans_query = """
        SELECT investorcode, amcid, foliono, trandate
        FROM MUTUALFUND_TRANSACTIONS
        
        """
        logger.debug("Fetching transaction data")
        trans_df = pd.read_sql(trans_query, engine)
        logger.info(f"Found {len(trans_df)} recent transactions")
        
        # 4. Validate data
        order_df, trans_df = validate_data(order_df, trans_df)
        
        # 5. Preprocess folio numbers
        logger.debug("Processing folio numbers")
        order_df['processed_folio'] = order_df['foliono'].apply(process_folio)
        trans_df['processed_folio'] = trans_df['foliono'].apply(process_folio)
        
        # 6. Process matches
        results = []
        match_count = 0
        
        logger.info(f"Matching {len(order_df)} orders against {len(trans_df)} transactions")
        
        for idx, order in order_df.iterrows():
            try:
                # Create matching conditions
                conditions = [
                    trans_df['investorcode'] == order['investorcode'],
                    trans_df['amcid'] == order['amcid']
                ]
                
                # Add folio condition if exists
                if pd.notna(order['processed_folio']):
                    conditions.append(
                        trans_df['processed_folio'] == order['processed_folio']
                    )
                
                # Combine conditions
                matching_trans = trans_df[pd.concat(conditions, axis=1).all(axis=1)]
                
                # Filter by registration date
                if pd.notna(order['regdate']):
                    matching_trans = matching_trans[matching_trans['trandate'] >= order['regdate']]
                
                # Prepare result
                result = {
                    'regno': order['regno'],
                    'investorcode': order['investorcode'],
                    'amcid': order['amcid'],
                    'foliono': order['foliono'],
                    'regdate': order['regdate'],
                    'has_match': False,
                    'match_count': 0,
                    'first_match_date': None,
                    'last_match_date': None
                }
                
                if not matching_trans.empty:
                    match_count += 1
                    result.update({
                        'has_match': True,
                        'match_count': len(matching_trans),
                        'first_match_date': matching_trans['trandate'].min(),
                        'last_match_date': matching_trans['trandate'].max()
                    })
                
                results.append(result)
                
                # Log progress
                if (idx + 1) % 100 == 0:
                    logger.debug(f"Processed {idx + 1} orders, found {match_count} matches")
                    
            except Exception as e:
                logger.error(f"Error processing order {order['regno']}: {str(e)}")
                continue
        
        # 7. Create results DataFrame
        result_df = pd.DataFrame(results)
        
        if result_df.empty:
            logger.warning("No matches found")
        else:
            logger.info(f"Found matches for {match_count}/{len(order_df)} orders")
        
        return result_df
        
    except Exception as e:
        logger.error(f"Processing failed: {str(e)}")
        logger.debug(traceback.format_exc())
        return pd.DataFrame()
    finally:
        if 'engine' in locals():
            engine.dispose()
        logger.info("Processing completed")

if __name__ == "__main__":
    try:
        # Configuration - replace with your actual credentials
        config = {
            'server': '192.168.100.55',
            'database': 'WEALTHONE',
            'username': 'aruhat',
            'password': 'aruhat'
        }
        
        # Run processing
        logger.info("Starting main execution")
        result = process_matches(**config)
        
        if result is not None and not result.empty:
            # Save results
            output_file = f"mf_matches_{datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"
            
            # Format dates for Excel
            date_cols = ['regdate', 'first_match_date', 'last_match_date']
            for col in date_cols:
                if col in result.columns:
                    result[col] = result[col].dt.strftime('%Y-%m-%d')
            
            # Save to Excel
            result.to_excel(output_file, index=False)
            
            # Print summary
            print("\nProcessing Results:")
            print(f"Total Orders Processed: {len(result)}")
            print(f"Orders With Matches: {result['has_match'].sum()}")
            print(f"Results saved to: {output_file}")
            
            # Show sample non-matches
            non_matches = result[~result['has_match']]
            if not non_matches.empty:
                print("\nSample Non-Matched Orders:")
                print(non_matches[['regno', 'investorcode', 'amcid']].head())
        else:
            print("Processing completed with no results. Check log file for details.")
        
    except Exception as e:
        print(f"Fatal error: {str(e)}\nCheck mf_matching_debug.log for details")

Processing completed with no results. Check log file for details.


In [1]:
import pandas as pd
from sqlalchemy import create_engine, exc
from datetime import datetime
import logging
import traceback

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='mf_amcid_matching_final.log',
    filemode='w'
)
logger = logging.getLogger(__name__)

def get_db_connection(server, database, username, password):
    """Create database connection with error handling"""
    try:
        conn_str = (
            f"mssql+pyodbc://{username}:{password}@{server}/{database}?"
            f"driver=ODBC+Driver+17+for+SQL+Server"
        )
        engine = create_engine(conn_str)
        # Test connection
        with engine.connect() as conn:
            conn.execute("SELECT 1")
        logger.info("Database connection successful")
        return engine
    except exc.SQLAlchemyError as e:
        logger.error(f"Connection failed: {str(e)}")
        raise

def validate_and_convert_data(order_df, trans_df):
    """Validate data and convert AMCID to consistent integer type"""
    logger.info("Validating and converting data")
    
    # Convert AMCID to integer
    for df in [order_df, trans_df]:
        if 'amcid' in df.columns:
            try:
                # First try direct conversion
                df['amcid'] = pd.to_numeric(df['amcid'], errors='raise').astype('Int64')
            except Exception as e:
                logger.warning(f"AMCID conversion warning: {str(e)}")
                try:
                    # Fallback to coerce with nulls
                    df['amcid'] = pd.to_numeric(df['amcid'], errors='coerce').astype('Int64')
                    if df['amcid'].isnull().any():
                        logger.warning(f"{df['amcid'].isnull().sum()} null AMCID values after conversion")
                except Exception as e:
                    logger.error(f"AMCID conversion failed: {str(e)}")
                    raise
    
    # Convert dates
    date_cols = {'order': 'regdate', 'trans': 'trandate'}
    for df_type, col in date_cols.items():
        df = order_df if df_type == 'order' else trans_df
        if col in df.columns:
            try:
                df[col] = pd.to_datetime(df[col], errors='coerce')
                if df[col].isnull().any():
                    logger.warning(f"{df[col].isnull().sum()} null {col} values after conversion")
            except Exception as e:
                logger.error(f"{col} conversion failed: {str(e)}")
                raise
    
    logger.info("Data validation and conversion complete")
    return order_df, trans_df

def process_mf_transactions(server, database, username, password):
    """Main processing function with proper AMCID handling"""
    try:
        logger.info("Starting MF transaction processing")
        
        # 1. Connect to database
        engine = get_db_connection(server, database, username, password)
        
        # 2. Fetch order data from mf_live
        order_query = """
        SELECT regno, investorcode, amcid, foliono, CAST(LEFT(Registrationdate, CHARINDEX('T', Registrationdate) - 1) AS DATE) regdate
        FROM MF_LIVE
        WHERE OrderStatus = 'Accepted'
        """
        logger.debug("Fetching order data")
        order_df = pd.read_sql(order_query, engine)
        logger.info(f"Retrieved {len(order_df)} orders")
        
        if order_df.empty:
            logger.error("No accepted orders found")
            return pd.DataFrame()

        # 3. Fetch transaction data
        trans_query = """
        SELECT investorcode, amcid, foliono, trandate
        FROM MUTUALFUND_TRANSACTIONS
        
        """
        logger.debug("Fetching transaction data")
        trans_df = pd.read_sql(trans_query, engine)
        logger.info(f"Retrieved {len(trans_df)} transactions")
        
        # 4. Validate and convert data
        order_df, trans_df = validate_and_convert_data(order_df, trans_df)
        
        # 5. Process matches
        results = []
        match_count = 0
        
        logger.info(f"Processing {len(order_df)} orders against {len(trans_df)} transactions")
        
        for idx, order in order_df.iterrows():
            try:
                # Create matching conditions
                conditions = [
                    trans_df['investorcode'] == order['investorcode'],
                    trans_df['amcid'] == order['amcid']
                ]
                
                # Add folio condition if exists
                if pd.notna(order['foliono']):
                    conditions.append(
                        trans_df['foliono'].astype(str) == order['foliono'].astype(str)
                    )
                
                # Combine conditions
                matching_trans = trans_df[pd.concat(conditions, axis=1).all(axis=1)]
                
                # Filter by registration date
                if pd.notna(order['regdate']):
                    matching_trans = matching_trans[matching_trans['trandate'] >= order['regdate']]
                
                # Prepare result
                result = {
                    'regno': order['regno'],
                    'investorcode': order['investorcode'],
                    'amcid': order['amcid'],
                    'foliono': order['foliono'],
                    'regdate': order['regdate'],
                    'has_match': False,
                    'match_count': 0,
                    'first_match_date': None,
                    'last_match_date': None
                }
                
                if not matching_trans.empty:
                    match_count += 1
                    result.update({
                        'has_match': True,
                        'match_count': len(matching_trans),
                        'first_match_date': matching_trans['trandate'].min(),
                        'last_match_date': matching_trans['trandate'].max()
                    })
                
                results.append(result)
                
                # Log progress every 100 records
                if (idx + 1) % 100 == 0:
                    logger.debug(f"Processed {idx + 1} orders, found {match_count} matches")
                    
            except Exception as e:
                logger.error(f"Error processing order {order['regno']}: {str(e)}")
                continue
        
        # 6. Create results DataFrame
        result_df = pd.DataFrame(results)
        
        if result_df.empty:
            logger.warning("No results generated")
        else:
            logger.info(f"Found {match_count} matches out of {len(order_df)} orders")
        
        return result_df
        
    except Exception as e:
        logger.error(f"Processing failed: {str(e)}")
        logger.debug(traceback.format_exc())
        return pd.DataFrame()
    finally:
        if 'engine' in locals():
            engine.dispose()
        logger.info("Processing completed")

if __name__ == "__main__":
    try:
        # Configuration
        config = {
            'server': '192.168.100.55',
            'database': 'Wealthone',
            'username': 'aruhat',
            'password': 'aruhat'
        }
        
        # Run processing
        logger.info("Starting main execution")
        result = process_mf_transactions(**config)
        
        if result is not None and not result.empty:
            # Save results
            output_file = f"mf_amcid_results_{datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"
            
            # Format dates for Excel
            date_cols = ['regdate', 'first_match_date', 'last_match_date']
            for col in date_cols:
                if col in result.columns:
                    result[col] = result[col].dt.strftime('%Y-%m-%d')
            
            # Save to Excel
            result.to_excel(output_file, index=False)
            
            # Print summary
            print("\nProcessing Summary:")
            print(f"Total Orders Processed: {len(result)}")
            print(f"Orders With Matches: {result['has_match'].sum()}")
            print(f"Results saved to: {output_file}")
            
            # Show sample non-matches for investigation
            non_matches = result[~result['has_match']]
            if not non_matches.empty:
                print("\nSample Non-Matched Orders:")
                print(non_matches[['regno', 'investorcode', 'amcid']].head())
        else:
            print("Processing completed with no results. Check log file for details.")
            
    except Exception as e:
        print(f"Fatal error occurred: {str(e)}\nSee mf_amcid_matching_final.log for details")

Processing completed with no results. Check log file for details.


In [5]:
import pandas as pd
from sqlalchemy import create_engine

def process_mf_transactions(server, database, username, password):
    """Main processing function with direct date handling"""
    try:
        # Establish database connection
        engine = create_engine(
            f"mssql+pyodbc://{username}:{password}@{server}/{database}?"
            "driver=ODBC+Driver+17+for+SQL+Server"
        )
        
        # Fetch order data with proper date conversion
        order_query = """
        SELECT 
            regno, 
            investorcode, 
            amcid, 
            foliono, 
            CAST(LEFT(registrationdate, CHARINDEX('T', registrationdate) - 1) AS DATE) AS regdate
        FROM mf_live
        WHERE orderstatus = 'Accepted'
        """
        order_df = pd.read_sql(order_query, engine)
        
        if order_df.empty:
            print("No accepted orders found")
            return pd.DataFrame()

        # Fetch transaction data
        trans_query = """
        SELECT 
            investorcode, 
            amcid, 
            foliono, 
            trandate
        FROM MUTUALFUND_TRANSACTIONS
        WHERE trandate >= DATEADD(month, -6, GETDATE())
        """
        trans_df = pd.read_sql(trans_query, engine)
        
        # Convert dates to datetime
        order_df['regdate'] = pd.to_datetime(order_df['regdate'])
        trans_df['trandate'] = pd.to_datetime(trans_df['trandate'])
        
        # Initialize results
        results = []
        
        # Process each order
        for _, order in order_df.iterrows():
            # Create matching conditions
            conditions = [
                trans_df['investorcode'] == order['investorcode'],
                trans_df['amcid'] == order['amcid']
            ]
            
            # Add folio condition if exists
            if pd.notna(order['foliono']):
                conditions.append(
                    trans_df['foliono'].astype(str) == order['foliono'].astype(str)
                )
            
            # Combine conditions
            matching_trans = trans_df[pd.concat(conditions, axis=1).all(axis=1)]
            
            # Filter by registration date
            if pd.notna(order['regdate']):
                matching_trans = matching_trans[matching_trans['trandate'] >= order['regdate']]
            
            # Prepare result
            result = {
                'regno': order['regno'],
                'investorcode': order['investorcode'],
                'amcid': order['amcid'],
                'foliono': order['foliono'],
                'regdate': order['regdate'],
                'has_match': False,
                'first_match_date': None
            }
            
            if not matching_trans.empty:
                result.update({
                    'has_match': True,
                    'first_match_date': matching_trans['trandate'].min()
                })
            
            results.append(result)
        
        # Create final DataFrame
        result_df = pd.DataFrame(results)
        
        return result_df
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return pd.DataFrame()
    finally:
        if 'engine' in locals():
            engine.dispose()

if __name__ == "__main__":
    # Configuration
    server = '192.168.100.55'
    database = 'WEALTHONE'
    username = 'aruhat'
    password = 'aruhat'
    
    # Process data
    result = process_mf_transactions(server, database, username, password)
    
    if not result.empty:
        # Save to Excel
        output_file = "mf_matching_results.xlsx"
        
        # Format dates
        result['regdate'] = result['regdate'].dt.strftime('%Y-%m-%d')
        result['first_match_date'] = result['first_match_date'].dt.strftime('%Y-%m-%d')
        
        result.to_excel(output_file, index=False)
        print(f"Results saved to {output_file}")
        
        # Print summary
        print(f"\nOrders processed: {len(result)}")
        print(f"Orders with matches: {result['has_match'].sum()}")
        
        # Show sample non-matches
        non_matches = result[~result['has_match']]
        if not non_matches.empty:
            print("\nSample non-matching orders:")
            print(non_matches[['regno', 'investorcode', 'amcid']].head())
    else:
        print("No results returned")

An error occurred: 'str' object has no attribute 'astype'
No results returned


In [19]:
import pandas as pd
from sqlalchemy import create_engine


server = '192.168.100.55'
database = 'WEALTHONE'
username = 'aruhat'
password = 'aruhat'

engine1 = create_engine(
            f"mssql+pyodbc://{username}:{password}@{server}/{database}?"
            "driver=ODBC+Driver+17+for+SQL+Server"
        ) 
def process_mf_transactions(server, database, username, password):
    """Main processing function with corrected folio handling"""
    try:
        # Establish database connection
        engine = create_engine(
            f"mssql+pyodbc://{username}:{password}@{server}/{database}?"
            "driver=ODBC+Driver+17+for+SQL+Server"
        )
        
        # Fetch order data with proper date conversion
        order_query = """
        SELECT 
            regno, 
            investorcode, 
            amcid, 
            foliono, 
            CAST(LEFT(registrationdate, CHARINDEX('T', registrationdate) - 1) AS DATE) AS regdate
        FROM mf_live
        WHERE orderstatus = 'Accepted'
        """
        order_df = pd.read_sql(order_query, engine)
        
        if order_df.empty:
            print("No accepted orders found")
            return pd.DataFrame()

        # Fetch transaction data
        trans_query = """
        SELECT 
            investorcode, 
            amcid, 
            CASE 
				WHEN CHARINDEX('/', FolioNo) > 0 THEN LEFT(FolioNo, CHARINDEX('/', FolioNo) - 1)
			ELSE FolioNo
			END AS foliono, 
            trandate
        FROM MUTUALFUND_TRANSACTIONS
     
        """
        trans_df = pd.read_sql(trans_query, engine)
        
        # Convert dates to datetime
        order_df['regdate'] = pd.to_datetime(order_df['regdate'])
        trans_df['trandate'] = pd.to_datetime(trans_df['trandate'])
        
        # Initialize results
        results = []
        
        # Process each order
        for _, order in order_df.iterrows():
            try:
                # Create matching conditions
                conditions = [
                    trans_df['investorcode'] == order['investorcode'],
                    trans_df['amcid'] == order['amcid']
                ]
                
                # Add folio condition if exists (corrected handling)
                if pd.notna(order['foliono']) and str(order['foliono']).strip() != '':
                    folio_str = str(order['foliono']).strip()
                    conditions.append(
                        trans_df['foliono'].apply(lambda x: str(x).strip() == folio_str
                    )
                    )
                # Combine conditions
                matching_trans = trans_df[pd.concat(conditions, axis=1).all(axis=1)]
                
                # Filter by registration date
                if pd.notna(order['regdate']):
                    matching_trans = matching_trans[matching_trans['trandate'] >= order['regdate']]
                
                # Prepare result
                result = {
                    'regno': order['regno'],
                    'investorcode': order['investorcode'],
                    'amcid': order['amcid'],
                    'foliono': order['foliono'],
                    'regdate': order['regdate'],
                    'has_match': False,
                    'first_match_date': None,
                    'match_count': 0
                }
                
                if not matching_trans.empty:
                    result.update({
                        'has_match': True,
                        'first_match_date': matching_trans['trandate'].min(),
                        'match_count': len(matching_trans)
                    })
                
                results.append(result)
                
            except Exception as e:
                print(f"Error processing order {order['regno']}: {str(e)}")
                continue
        
        # Create final DataFrame
        result_df = pd.DataFrame(results)
        
        return result_df
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return pd.DataFrame()
    finally:
        if 'engine' in locals():
            engine.dispose()

if __name__ == "__main__":
    # Configuration
    server = '192.168.100.55'
    database = 'WEALTHONE'
    username = 'aruhat'
    password = 'aruhat'
    
    # Process data
    result = process_mf_transactions(server, database, username, password)
    
    if not result.empty:
        # Save to Excel
        output_file = "mf_matching_results.xlsx"
        
        # Format dates
        result['regdate'] = result['regdate'].dt.strftime('%Y-%m-%d')
        result['first_match_date'] = result['first_match_date'].dt.strftime('%Y-%m-%d')
        
        result.to_excel(output_file, index=False)
        result.to_sql('INVESTOR_MATCHES', con=engine1, if_exists="append", index=False)
        print(f"Results saved to {output_file}")
        
        # Print summary
        print(f"\nOrders processed: {len(result)}")
        print(f"Orders with matches: {result['has_match'].sum()}")
        
        # Show sample non-matches
        non_matches = result[~result['has_match']]
        if not non_matches.empty:
            print("\nSample non-matching orders:")
            print(non_matches[['regno', 'investorcode', 'amcid', 'foliono']].head())
            
    else:
        print("No results returned")

NameError: name 'engine' is not defined

In [17]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

def process_mf_transactions(server, database, username, password):
    """Main processing function with SQL insertion"""
    try:
        # Establish database connection
        engine = create_engine(
            f"mssql+pyodbc://{username}:{password}@{server}/{database}?"
            "driver=ODBC+Driver+17+for+SQL+Server"
        )
        
        # Fetch order data with proper date conversion
        order_query = """
        SELECT 
            regno, 
            investorcode, 
            amcid, 
            foliono, 
            CAST(LEFT(registrationdate, CHARINDEX('T', registrationdate) - 1) AS DATE) AS regdate
        FROM mf_live
        WHERE orderstatus = 'Accepted'
        """
        order_df = pd.read_sql(order_query, engine)
        
        if order_df.empty:
            print("No accepted orders found")
            return pd.DataFrame()

        # Fetch transaction data
        trans_query = """
        SELECT 
            investorcode, 
            amcid, 
            foliono, 
            trandate
        FROM MUTUALFUND_TRANSACTIONS
        WHERE trandate >= DATEADD(month, -6, GETDATE())
        """
        trans_df = pd.read_sql(trans_query, engine)
        
        # Convert dates to datetime
        order_df['regdate'] = pd.to_datetime(order_df['regdate'])
        trans_df['trandate'] = pd.to_datetime(trans_df['trandate'])
        
        # Initialize results
        results = []
        
        # Process each order
        for _, order in order_df.iterrows():
            try:
                # Create matching conditions
                conditions = [
                    trans_df['investorcode'] == order['investorcode'],
                    trans_df['amcid'] == order['amcid']
                ]
                
                # Add folio condition if exists
                if pd.notna(order['foliono']) and str(order['foliono']).strip() != '':
                    folio_str = str(order['foliono']).strip()
                    conditions.append(
                        trans_df['foliono'].apply(lambda x: str(x).strip() == folio_str)
                    )
                
                # Combine conditions
                matching_trans = trans_df[pd.concat(conditions, axis=1).all(axis=1)]
                
                # Filter by registration date
                if pd.notna(order['regdate']):
                    matching_trans = matching_trans[matching_trans['trandate'] >= order['regdate']]
                
                # Prepare result
                result = {
                    'regno': order['regno'],
                    'investorcode': order['investorcode'],
                    'amcid': order['amcid'],
                    'foliono': order['foliono'],
                    'regdate': order['regdate'],
                    'has_match': False,
                    'first_match_date': None,
                    'match_count': 0,
                    'processing_date': datetime.now()
                }
                
                if not matching_trans.empty:
                    result.update({
                        'has_match': True,
                        'first_match_date': matching_trans['trandate'].min(),
                        'match_count': len(matching_trans)
                    })
                
                results.append(result)
                
            except Exception as e:
                print(f"Error processing order {order['regno']}: {str(e)}")
                continue
        
        # Create final DataFrame
        result_df = pd.DataFrame(results)
        
        if not result_df.empty:
            # Insert results into SQL table
            result_df.to_sql(
                name='mf_matching_results',  # Your target table name
                con=engine,
                if_exists='append',  # Append to existing data
                index=False,
                dtype={
                    'regno': pd.StringDtype(),
                    'investorcode': pd.StringDtype(),
                    'amcid': pd.Int64Dtype(),
                    'foliono': pd.StringDtype(),
                    'regdate': pd.Timestamp,
                    'has_match': pd.StringDtype(),
                    'first_match_date': pd.Timestamp,
                    'match_count': pd.Int64Dtype(),
                    'execution_time': pd.Timestamp
                }
            )
            print("Results successfully inserted into SQL table")
        
        return result_df
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return pd.DataFrame()
    finally:
        if 'engine' in locals():
            engine.dispose()

if __name__ == "__main__":
    # Configuration
    server = '192.168.100.55'
    database = 'Wealthone'
    username = 'aruhat'
    password = 'aruhat'
    
    # Process data
    result = process_mf_transactions(server, database, username, password)
    
    if not result.empty:
        # Save to Excel
        output_file = "mf_matching_results.xlsx"
        
        # Format dates
        result['regdate'] = result['regdate'].dt.strftime('%Y-%m-%d')
        result['first_match_date'] = result['first_match_date'].dt.strftime('%Y-%m-%d')
        
        result.to_excel(output_file, index=False)
        print(f"Results saved to {output_file}")
        
        # Print summary
        print(f"\nOrders processed: {len(result)}")
        print(f"Orders with matches: {result['has_match'].sum()}")
        
        # Show sample non-matches
        non_matches = result[~result['has_match']]
        if not non_matches.empty:
            print("\nSample non-matching orders:")
            print(non_matches[['regno', 'investorcode', 'amcid', 'foliono']].head())
    else:
        print("No results returned")

An error occurred: The type of regno is not a SQLAlchemy type
No results returned


In [25]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

# Database configuration
server = '192.168.100.55'
database = 'WEALTHONE'
username = 'aruhat'
password = 'aruhat'

# Create database connection
engine = create_engine(
    f"mssql+pyodbc://{username}:{password}@{server}/{database}?"
    "driver=ODBC+Driver+17+for+SQL+Server"
)

try:
    # Fetch order data with date conversion
    order_df = pd.read_sql("""
        SELECT 
            regno, 
            investorcode, 
            amcid, 
            foliono, 
            CAST(LEFT(registrationdate, CHARINDEX('T', registrationdate) - 1) AS DATE) AS regdate
        FROM mf_live
        WHERE orderstatus = 'Accepted'
    """, engine)
    
    if order_df.empty:
        print("No accepted orders found")
    else:
        # Fetch transaction data
        trans_df = pd.read_sql("""
            SELECT 
                investorcode, 
                amcid, 
                foliono, 
                trandate
            FROM MUTUALFUND_TRANSACTIONS
            
        """, engine)
        
        # Convert dates
        order_df['regdate'] = pd.to_datetime(order_df['regdate'])
        trans_df['trandate'] = pd.to_datetime(trans_df['trandate'])
        
        # Process matches
        results = []
        for _, order in order_df.iterrows():
            # Create matching conditions
            conditions = [
                trans_df['investorcode'] == order['investorcode'],
                trans_df['amcid'] == order['amcid']
            ]
            
            # Add folio condition if exists
            if pd.notna(order['foliono']) and str(order['foliono']).strip():
                folio_str = str(order['foliono']).strip()
                conditions.append(trans_df['foliono'].apply(lambda x: str(x).strip() == folio_str))
            
            # Find matches
            matches = trans_df[pd.concat(conditions, axis=1).all(axis=1)]
            if pd.notna(order['regdate']):
                matches = matches[matches['trandate'] >= order['regdate']]
            
            # Store results
            result = {
                'regno': order['regno'],
                'investorcode': order['investorcode'],
                'amcid': order['amcid'],
                'foliono': order['foliono'],
                'regdate': order['regdate'],
                'has_match': not matches.empty,
                'first_match_date': matches['trandate'].min() if not matches.empty else None,
                'execution_time': datetime.now()
            }
            results.append(result)
        
        # Create DataFrame and save to SQL
        result_df = pd.DataFrame(results)
        if not result_df.empty:
            result_df.to_sql(
                'INVESTOR_MATCHES',
                engine,
                if_exists='append',
                index=False
            )
            print(f"Inserted {len(result_df)} records into SQL")
            
            # Also save to Excel
            result_df.to_excel("results.xlsx", index=False)
            print("Saved results to Excel")

except Exception as e:
    print(f"Error occurred: {str(e)}")
finally:
    engine.dispose()
    print("Processing completed")

Inserted 1442 records into SQL
Saved results to Excel
Processing completed
